# 2D-Phase-Painter-ForLC
### 与闪电晶体号对接的2维相图绘制
最后更新：2022.04.06 戴以恒  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from multiprocessing import Pool
from IPython.display import clear_output as clear
import time
import math
import gc
c_time = time.strftime("%Y%m%d_%H%M%S", time.localtime())
c_time_m = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

In [ ]:
# 参数
# ======== System Setup ========
Version = 'SVC-V1.3b-Alpha-c1c4-Grain'
EPOCH = 30 # 每个c1,c2条件绘制1张相图需要EPOCH轮，注意合理设置这些参数，防止运行时间暴增
C1_UPPER = 3.00   # c1绘制相图的浓度上限，浓度单位为mM，即mmol/L
C1_LOWER = 0.00   # c1绘制相图的浓度下限，浓度单位为mM，即mmol/L
C1_NUM = 51       # 在上下限之间绘制的点数
C4_UPPER = 60.0
C4_LOWER = 0.00
C4_NUM = 61
# C2_UPPER = 3.00
# C2_LOWER = 0.00
# C2_NUM = 151
# ======== Fit Data Input ========
INPUT_X_C = 'Features_499_5_c.csv'
INPUT_Y_C = 'Values_Red_499_R.csv'

INPUT_X_P = 'Features_68_5_c.csv'
INPUT_Y_P = 'Values_YF_68.csv'

INPUT_TITLE = 'Title_5_c.csv'
# ======== Other Fitting Settings ========
TRAIN_TEST_SPLIT_1 = 0.85
TRAIN_TEST_SPLIT_2 = 0.75
# ======== Draw Settings ========
C3_UPPER = 80.0   # c3绘制相图的浓度上限，浓度单位为mM，即mmol/L
C3_LOWER = 0.0   # c3绘制相图的浓度下限，浓度单位为mM，即mmol/L
C3_NUM = 30001       # 在上下限之间绘制的点数
# 由于浓度的自由度只有3，实际上c2由c1、c2、c4共同决定。
# C4_UPPER = 17.0
# C4_LOWER = 2.0
# C4_NUM = 51
C5_PRODUCT_TIME = 1.949  # 2.601代表5:1加水，1.949代表5:0.75加水
# ======== Sampling Settings ========
C0 = [3.72, 4.966, 100.0, 60.0]     # 原始数据C1的溶液浓度，单位为M，即mmol/L

In [ ]:
X_C = np.loadtxt(INPUT_X_C, delimiter=',')
title = np.loadtxt(INPUT_TITLE, dtype='str', delimiter=',', comments='!')
y_c = np.loadtxt(INPUT_Y_C, delimiter=',', dtype=float)
X_PP = np.loadtxt(INPUT_X_P, delimiter=',')
y_pp = np.loadtxt(INPUT_Y_P, delimiter=',', dtype=float)
print('X:', X_C.shape, '   y:', y_c.shape)

In [ ]:
from sklearn import model_selection
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
import joblib

In [ ]:
import os
from pathlib import Path
DIR = '2D-Phase-Painter-LC_'+Version+'_'+c_time
os.mkdir(DIR)
RECORD_NAME = Path('.', DIR, '2DPS-LC_00_Record_'+c_time+'.txt')
f1 = open(RECORD_NAME, 'w')
f1.write('2DPS-LC Starts at '+time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())+'\n\n')
DIR2 = Path('.', DIR, 'TEMP')
os.mkdir(DIR2)

In [ ]:
# 生成c1、c2格点：
l_c1 = np.linspace(C1_LOWER, C1_UPPER, C1_NUM)
l_c4 = np.linspace(C4_LOWER, C4_UPPER, C4_NUM)
c_list = []
idx_list = []
for i in range(len(l_c1)):
    for j in range(len(l_c4)):
        c_list.append([l_c1[i], l_c4[j]])
        idx_list.append([i, j])
c_list = np.array(c_list)
idx_list = np.array(idx_list)

In [ ]:
# 大循环
out = []
a_m_1 = np.zeros((len(l_c1), len(l_c4)))
a_m_rela_1 = np.zeros((len(l_c1), len(l_c4)))
pp_num = np.zeros((len(l_c1), len(l_c4)))
TEMP_NAME = None
for kk in range(c_list.shape[0]):
    TARGET_C1 = c_list[kk, 0]
    TARGET_C4 = c_list[kk, 1]
    print('Epoch', kk+1, 'Begin:  c1:', TARGET_C1, '  c4:', TARGET_C4)
    f1.write('  Epoch '+str(kk+1)+' started at '+time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())+'\n')
    if kk != 0:
        os.remove(TEMP_NAME)
    TEMP_NAME = Path(DIR2, '2DPS_Epoch_'+str(kk+1).zfill(4)+'-'+str(c_list.shape[0]).zfill(4)+'_'+time.strftime("%Y%m%d_%H%M%S", time.localtime())+'.txt')
    f2 = open(TEMP_NAME, 'w')
    f2.close()
    # 生成格点数据集
    c3_l = np.linspace(C3_LOWER, C3_UPPER, C3_NUM)
    # c4_l = np.linspace(C4_LOWER, C4_UPPER, C4_NUM)
    X_p_C = []
    for i in range(len(c3_l)):
        ttt = 1 - TARGET_C1/C0[0] - TARGET_C4/C0[3] - c3_l[i]/C0[2]
        if ttt>=0:
            c2 = C0[1]*ttt
            X_p_C.append([TARGET_C1, c2, c3_l[i], TARGET_C4, C5_PRODUCT_TIME*TARGET_C1])
    pp_num[idx_list[kk, 0], idx_list[kk, 1]] = len(X_p_C)
    X_p_C = np.array(X_p_C)
    if X_p_C.shape[0] > 0:
        # 预测有无还原现象
        clf = SVC(kernel='rbf', gamma=np.exp(-5.993538850618485), C=13.059715322026376, verbose=1, max_iter=-1, cache_size=40960, probability=True)
        paras = clf.get_params()
        point = round(X_C.shape[0]*TRAIN_TEST_SPLIT_1)
        y_p_m = []
        y_p_c = np.zeros((X_p_C.shape[0], 1))
        acc_list = []
        for _ in range(EPOCH):
            permutation = np.random.permutation(y_c.shape[0])
            train_idx = permutation[:point]
            test_idx = permutation[point:]
            X_train = X_C[train_idx, :]
            y_train = y_c[train_idx]
            X_test = X_C[test_idx, :]
            y_test = y_c[test_idx]
            clf_new = SVC()
            for k, v in paras.items():
                clf_new.set_params(**{k: v})
            # 拟合模型
            clf_new.fit(X_train, y_train)
            # 计算损失
            y_pred = clf_new.predict(X_test)
            acc_count = 0
            for i in range(X_test.shape[0]):
                if y_pred[i]==y_test[i]:
                    acc_count += 1
            acc = acc_count*100/X_test.shape[0]
            y_p = clf_new.predict_proba(X_p_C)
            y_p_l = y_p[:, 1].flatten().tolist()
            acc_list.append(acc)
            y_p_m.append(y_p_l)
        y_p_m = np.array(y_p_m)
        y_p_m = y_p_m.transpose()
        for i in range(X_p_C.shape[0]):
            y_p_c[i, 0] = np.mean(y_p_m[i, :])
        del y_p_m
        # 加权计算
        y_p_c_l = []
        for i in range(y_p_c.shape[0]):
            y_p_c_l.append(1/(1+np.exp(15-30*y_p_c[i, 0])))
        X_p_t1 = []
        for i in range(X_p_C.shape[0]):
            if y_p_c_l[i] > 0.5:
                X_p_t1.append(X_p_C[i, :].flatten().tolist())
        X_p_t1 = np.array(X_p_t1)
        if X_p_t1.shape[0] > 0:
            # 预测是否产生黄色荧光
            clf = SVC(kernel='rbf', gamma=np.exp(-6.964045492214664), C=13.589042049663163, verbose=1, max_iter=-1, cache_size=40960, probability=True)
            paras = clf.get_params()
            point = round(X_PP.shape[0]*TRAIN_TEST_SPLIT_1)
            y_p_m = []
            y_p_t1 = np.zeros((X_p_t1.shape[0], 1))
            acc_list = []
            for _ in range(EPOCH):
                permutation = np.random.permutation(y_pp.shape[0])
                train_idx = permutation[:point]
                test_idx = permutation[point:]
                X_train = X_PP[train_idx, :]
                y_train = y_pp[train_idx]
                X_test = X_PP[test_idx, :]
                y_test = y_pp[test_idx]
                clf_new = SVC()
                for k, v in paras.items():
                    clf_new.set_params(**{k: v})
                # 拟合模型
                clf_new.fit(X_train, y_train)
                # 计算损失
                y_pred = clf_new.predict(X_test)
                acc_count = 0
                for i in range(X_test.shape[0]):
                    if y_pred[i]==y_test[i]:
                        acc_count += 1
                acc = acc_count*100/X_test.shape[0]
                y_p = clf_new.predict_proba(X_p_t1)
                y_p_l = y_p[:, 1].flatten().tolist()
                acc_list.append(acc)
                y_p_m.append(y_p_l)
            y_p_m = np.array(y_p_m)
            y_p_m = y_p_m.transpose()
            for i in range(X_p_t1.shape[0]):
                y_p_t1[i, 0] = np.mean(y_p_m[i, :])
            clear()
            del y_p_m
            y_p_t1_l = []
            for i in range(y_p_t1.shape[0]):
                y_p_t1_l.append(1/(1+np.exp(15-30*y_p_t1[i, 0])))
            out.append(sum(y_p_t1_l))
            a_m_1[idx_list[kk, 0], idx_list[kk, 1]] = sum(y_p_t1_l)
            a_m_rela_1[idx_list[kk, 0], idx_list[kk, 1]] = sum(y_p_t1_l)/len(y_p_t1_l)
        else:
            out.append(0)
            a_m_1[idx_list[kk, 0], idx_list[kk, 1]] = 0
            a_m_rela_1[idx_list[kk, 0], idx_list[kk, 1]] = 0
    else:
        out.append(0)
        a_m_1[idx_list[kk, 0], idx_list[kk, 1]] = 0
        a_m_rela_1[idx_list[kk, 0], idx_list[kk, 1]] = 0

In [ ]:
# 保存数据
f1.close()
save_name = '2DPS_01a_Red_Matrix_Abso_'+str(a_m_1.shape[0])+'_'+str(a_m_1.shape[1])+'.csv'
save_name = Path('.', DIR, save_name)
np.savetxt(save_name, a_m_1, fmt='%s', delimiter=',')
save_name = '2DPS_01b_Red_Matrix_Rela_'+str(a_m_rela_1.shape[0])+'_'+str(a_m_rela_1.shape[1])+'.csv'
save_name = Path('.', DIR, save_name)
np.savetxt(save_name, a_m_rela_1, fmt='%s', delimiter=',')
save_name = '2DPS_01c_DP_Num_'+str(pp_num.shape[0])+'_'+str(pp_num.shape[1])+'.csv'
save_name = Path('.', DIR, save_name)
np.savetxt(save_name, pp_num, fmt='%s', delimiter=',')

In [ ]:
# 绘制针对553的搜索图：
x_t = []
y_t = []
x_ticks = []
y_ticks = []
for i in range(len(l_c1)):
    if i % 2 == 0:
        x_t.append(i)
        if i%4 == 0:
            x_ticks.append(str(round(l_c1[i], 3)))
        else:
            x_ticks.append('')
for i in range(len(l_c4)):
    if i % 2 == 0:
        y_t.append(i)
        if i%4 == 0:
            y_ticks.append(str(round(l_c4[i], 3)))
        else:
            y_ticks.append('')

fig = plt.figure(figsize=(7.2, 7.8), dpi=300)
ax = fig.add_axes([0.12, 0.11, 0.84, 0.72])
sc = ax.imshow(a_m_1.transpose(), cmap='RdYlGn', origin='lower', aspect='auto', zorder=5, vmin=0.0)
ax.plot([0, C0[0]*(C1_NUM-1)/(C1_UPPER-C1_LOWER)], [C0[3]*(C4_NUM-1)/(C4_UPPER-C4_LOWER), 0], 
        linestyle=':', linewidth=3.0, zorder=10, color='#87CEEB')
ax.set_xlim(0, C1_NUM-1)
ax.set_ylim(0, C4_NUM-1)
ax.set_xticks(x_t)
ax.set_yticks(y_t)
ax.set_xticklabels(x_ticks, rotation=30)
ax.set_yticklabels(y_ticks)
ax.set_xlabel(title[0], fontsize=17)
ax.set_ylabel(title[3], fontsize=17)
plt.suptitle('2D Phase Searching of Reduction by C\n'+
             'C Range:c1:'+str(C1_LOWER)+'~'+str(C1_UPPER)+'mM    c4:'+str(C4_LOWER)+'~'+str(C4_UPPER)+'mM\n'+
             'C Range:c3:'+str(C3_LOWER)+'~'+str(C3_UPPER)+'mM\n '+
             'c5 Product Time:'+str(C5_PRODUCT_TIME)+'\n', fontsize=18)
cb = plt.colorbar(sc)
cb.set_label('Absolute Area of Reduction', fontsize=17)
save_name = '2DPS_02a_2D-Phase-Searching_Red_Abosulte_Area.png'
save_name = Path('.', DIR, save_name)
plt.savefig(save_name)

fig = plt.figure(figsize=(7.2, 7.8), dpi=300)
ax = fig.add_axes([0.12, 0.11, 0.84, 0.72])
sc = ax.imshow(a_m_rela_1.transpose(), cmap='RdYlGn', origin='lower', aspect='auto', zorder=5, vmin=0.0)
ax.plot([0, C0[0]*(C1_NUM-1)/(C1_UPPER-C1_LOWER)], [C0[3]*(C4_NUM-1)/(C4_UPPER-C4_LOWER), 0], 
        linestyle=':', linewidth=3.0, zorder=10, color='#87CEEB')
ax.set_xlim(0, C1_NUM-1)
ax.set_ylim(0, C4_NUM-1)
ax.set_xticks(x_t)
ax.set_yticks(y_t)
ax.set_xticklabels(x_ticks, rotation=30)
ax.set_yticklabels(y_ticks)
ax.set_xlabel(title[0], fontsize=17)
ax.set_ylabel(title[3], fontsize=17)
plt.suptitle('2D Phase Searching of Reduction by C\n'+
             'C Range:c1:'+str(C1_LOWER)+'~'+str(C1_UPPER)+'mM    c4:'+str(C4_LOWER)+'~'+str(C4_UPPER)+'mM\n'+
             'C Range:c3:'+str(C3_LOWER)+'~'+str(C3_UPPER)+'mM\n'+
             'c5 Product Time:'+str(C5_PRODUCT_TIME)+'\n', fontsize=18)
cb = plt.colorbar(sc)
cb.set_label('Relative Area of Reduction', fontsize=17)
save_name = '2DPS_02b_2D-Phase-Searching_Red_Relative_Area.png'
save_name = Path('.', DIR, save_name)
plt.savefig(save_name)

In [ ]:
red_0_m = []
red_1_m = []
yf_0_m = []
yf_1_m = []
for i in range(X_C.shape[0]):
    if y_c[i, ] == 0:
        red_0_m.append([X_C[i, 0]*(C1_NUM-1)/(C1_UPPER-C1_LOWER), X_C[i, 3]*(C4_NUM-1)/(C4_UPPER-C4_LOWER)])
    else:
        red_1_m.append([X_C[i, 0]*(C1_NUM-1)/(C1_UPPER-C1_LOWER), X_C[i, 3]*(C4_NUM-1)/(C4_UPPER-C4_LOWER)])
for i in range(X_PP.shape[0]):
    if y_pp[i, ] == 0:
        yf_0_m.append([X_PP[i, 0]*(C1_NUM-1)/(C1_UPPER-C1_LOWER), X_PP[i, 3]*(C4_NUM-1)/(C4_UPPER-C4_LOWER)])
    else:
        yf_1_m.append([X_PP[i, 0]*(C1_NUM-1)/(C1_UPPER-C1_LOWER), X_PP[i, 3]*(C4_NUM-1)/(C4_UPPER-C4_LOWER)])
red_0_m = np.array(red_0_m)
red_1_m = np.array(red_1_m)
yf_0_m = np.array(yf_0_m)
yf_1_m = np.array(yf_1_m)

In [ ]:
fig = plt.figure(figsize=(7.2, 7.8), dpi=300)
ax = fig.add_axes([0.12, 0.11, 0.84, 0.72])
sc = ax.imshow(pp_num.transpose(), cmap='BuPu', origin='lower', aspect='auto', zorder=5, vmin=0.0)
ax.plot([0, C0[0]*(C1_NUM-1)/(C1_UPPER-C1_LOWER)], [C0[3]*(C4_NUM-1)/(C4_UPPER-C4_LOWER), 0], 
        linestyle=':', linewidth=3.0, zorder=10, color='#87CEEB')
ax.scatter(red_0_m[:, 0], red_0_m[:, 1], edgecolors='black', linewidths=2, c='#C0C0C0', zorder=15)
ax.scatter(red_1_m[:, 0], red_1_m[:, 1], edgecolors='black', linewidths=2, c='#ADFF2F', zorder=15)
ax.scatter(yf_0_m[:, 0], yf_0_m[:, 1], edgecolors='#696969', linewidths=2, c='#F5FFFA', zorder=15)
ax.scatter(yf_1_m[:, 0], yf_1_m[:, 1], edgecolors='#696969', linewidths=2, c='#FF8C00', zorder=15)
ax.set_xlim(0, C1_NUM-1)
ax.set_ylim(0, C4_NUM-1)
ax.set_xticks(x_t)
ax.set_yticks(y_t)
ax.set_xticklabels(x_ticks, rotation=30)
ax.set_yticklabels(y_ticks)
ax.set_xlabel(title[0], fontsize=17)
ax.set_ylabel(title[3], fontsize=17)
plt.suptitle('2D Phase Searching of DP Count by C\n'+
             'C Range:c1:'+str(C1_LOWER)+'~'+str(C1_UPPER)+'mM    c4:'+str(C4_LOWER)+'~'+str(C4_UPPER)+'mM\n'+
             'C Range:c3:'+str(C3_LOWER)+'~'+str(C3_UPPER)+'mM\n'+
             'c5 Product Time:'+str(C5_PRODUCT_TIME)+'\n', fontsize=18)
cb = plt.colorbar(sc)
cb.set_label('Number of Data Points', fontsize=17)
save_name = '2DPS_02c_2D-Phase-Searching_DP_Num.png'
save_name = Path('.', DIR, save_name)
plt.savefig(save_name)